<div style="border: 2px solid #8A9AD0; margin: 1em 0.2em; padding: 0.5em;">

# Scripting Galaxy using the API and BioBlend

by [Nicola Soranzo](https://training.galaxyproject.org/hall-of-fame/nsoranzo/), [Clare Sloggett](https://training.galaxyproject.org/hall-of-fame/claresloggett/), [Nitesh Turaga](https://training.galaxyproject.org/hall-of-fame/nturaga/), [Helena Rasche](https://training.galaxyproject.org/hall-of-fame/hexylena/)

CC-BY licensed content from the [Galaxy Training Network](https://training.galaxyproject.org/)

**Objectives**

- What is a REST API?
- How to interact with Galaxy programmatically?
- Why and when should I use BioBlend?

**Objectives**

- Interact with Galaxy via BioBlend.

**Time Estimation: 2h**
</div>


<blockquote class="agenda" style="border: 2px solid #86D486;display: none; margin: 1em 0.2em">
<div id="agenda" class="box-title" aria-label="agenda box: ">Agenda</div>
<p>In this tutorial, we will cover:</p>
<ol id="markdown-toc">
<li><a href="#interacting-with-histories-in-galaxy-api" id="markdown-toc-interacting-with-histories-in-galaxy-api">Interacting with histories in Galaxy API</a></li>
</ol>
</blockquote>
<h1 id="interacting-with-histories-in-galaxy-api">Interacting with histories in Galaxy API</h1>
<p>We are going to use the <a href="https://requests.readthedocs.io/">requests</a> Python library to communicate via HTTP with the Galaxy server. To start, let’s define the connection parameters.</p>
<p><strong>You need to insert the API key for your Galaxy server in the cell below</strong>:</p>
<ol>
<li>Open the Galaxy server in another browser tab</li>
<li>Click on “User” on the top menu, then “Preferences”</li>
<li>Click on “Manage API key”</li>
<li>Generate an API key if needed, then copy the alphanumeric string and paste it as the value of the <code style="color: inherit">api_key</code> variable below.</li>
</ol>


In [ ]:
import json
from pprint import pprint
from urllib.parse import urljoin

import requests

server = 'https://usegalaxy.eu/'
api_key = ''
base_url = urljoin(server, 'api')
base_url

<p>We now make a GET request to retrieve all histories owned by a user:</p>


In [ ]:
headers = {"Content-Type": "application/json", "x-api-key": api_key}
r = requests.get(base_url + "/histories", headers=headers)
print(r.text)
hists = r.json()
pprint(hists)

<p>As you can see, GET requests in Galaxy API return JSON strings, which need to be <strong>deserialized</strong> into Python data structures. In particular, GETting a resource collection returns a list of dictionaries.</p>
<p>Each dictionary returned when GETting a resource collection gives basic info about a resource, e.g. for a history you have:</p>
<ul>
<li><code style="color: inherit">id</code>: the unique <strong>identifier</strong> of the history, needed for all specific requests about this resource</li>
<li><code style="color: inherit">name</code>: the name of this history as given by the user</li>
<li><code style="color: inherit">deleted</code>: whether the history has been deleted.</li>
</ul>
<p>There is no readily-available filtering capability, but it’s not difficult to filter histories <strong>by name</strong>:</p>


In [ ]:
pprint([_ for _ in hists if _['name'] == 'Unnamed history'])

<p>If you are interested in more <strong>details</strong> about a given resource, you just need to append its <code style="color: inherit">id</code> to the previous collection request, e.g. to the get more info for a history:</p>


In [ ]:
hist0_id = hists[0]['id']
print(hist0_id)
r = requests.get(base_url + "/histories/" + hist0_id, headers=headers)
pprint(r.json())

<p>As you can see, there are much more entries in the returned dictionary, e.g.:</p>
<ul>
<li><code style="color: inherit">create_time</code></li>
<li><code style="color: inherit">size</code>: total disk space used by the history</li>
<li><code style="color: inherit">state_ids</code>: ids of history datasets for each possible state.</li>
</ul>
<p>To get the list of <strong>datasets contained</strong> in a history, simply append <code style="color: inherit">/contents</code> to the previous resource request.</p>


In [ ]:
r = requests.get(base_url + "/histories/" + hist0_id + "/contents", headers=headers)
hdas = r.json()
pprint(hdas)

<p>The dictionaries returned when GETting the history content give basic info about each dataset, e.g.: <code style="color: inherit">id</code>, <code style="color: inherit">name</code>, <code style="color: inherit">deleted</code>, <code style="color: inherit">state</code>, <code style="color: inherit">url</code>…</p>
<p>To get the details about a specific dataset, you can use the <code style="color: inherit">datasets</code> controller:</p>


In [ ]:
hda0_id = hdas[0]['id']
print(hda0_id)
r = requests.get(base_url + "/datasets/" + hda0_id, headers=headers)
pprint(r.json())

<p>Some of the interesting additional dictionary entries are:</p>
<ul>
<li><code style="color: inherit">create_time</code></li>
<li><code style="color: inherit">creating job</code>: id of the job which created this dataset</li>
<li><code style="color: inherit">download_url</code>: URL to download the dataset</li>
<li><code style="color: inherit">file_ext</code>: the Galaxy data type of this dataset</li>
<li><code style="color: inherit">file_size</code></li>
<li><code style="color: inherit">genome_build</code>: the genome build (dbkey) associated to this dataset.</li>
</ul>
<p><strong>New resources</strong> are created with POST requests. The uploaded <strong>data needs to be serialized</strong> in a JSON string. For example, to create a new history:</p>


In [ ]:
data = {'name': 'New history'}
r = requests.post(base_url + "/histories", data=json.dumps(data), headers=headers)
new_hist = r.json()
pprint(new_hist)

<p>The return value of a POST request is a dictionary with detailed info about the created resource.</p>
<p>To <strong>update</strong> a resource, make a PUT request, e.g. to change the history name:</p>


In [ ]:
data = {'name': 'Updated history'}
r = requests.put(base_url + "/histories/" + new_hist["id"], json.dumps(data), headers=headers)
print(r.status_code)
pprint(r.json())

<p>The return value of a PUT request is usually a dictionary with detailed info about the updated resource.</p>
<p>Finally to <strong>delete</strong> a resource, make a DELETE request, e.g.:</p>


In [ ]:
r = requests.delete(base_url + "/histories/" + new_hist["id"], headers=headers)
print(r.status_code)

<h2 id="exercise-galaxy-api">Exercise: Galaxy API</h2>
<p><strong>Goal</strong>: Upload a file to a new history, import a workflow and run it on the uploaded dataset.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-initialise" class="box-title" aria-label="question box: Initialise"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Initialise</div>
<p>First, define the connection parameters. What variables do you need?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution" class="box-title"><button type="button" aria-controls="solution-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">import json
from pprint import pprint
from urllib.parse import urljoin

import requests

server = 'https://usegalaxy.eu/'
api_key = ''
base_url = urljoin(server, 'api')
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-new-history" class="box-title" aria-label="question box: New History"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: New History</div>
<p>Next, create a new Galaxy history via POST to the correct API.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-1" class="box-title"><button type="button" aria-controls="solution-1-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">headers = {"Content-Type": "application/json", "x-api-key": api_key}
data = {"name": "New history"}
r = requests.post(base_url + "/histories", data=json.dumps(data), headers=headers)
new_hist = r.json()
pprint(new_hist)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-upload-a-dataset" class="box-title" aria-label="question box: Upload a dataset"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Upload a dataset</div>
<p><strong>Upload</strong> the local file <code style="color: inherit">1.txt</code> to the new history. You need to run the special <code style="color: inherit">upload1</code> tool by making a <code style="color: inherit">POST</code> request to <code style="color: inherit">/api/tools</code>. You don’t need to pass any inputs to it apart from attaching the file as <code style="color: inherit">files_0|file_data</code>. Also, note that when attaching a file you need to drop <code style="color: inherit">Content-Type</code> from the request headers.</p>
<p>You can obtain the <code style="color: inherit">1.txt</code> file from the following URL, you’ll need to download it first.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">https://raw.githubusercontent.com/nsoranzo/bioblend-tutorial/main/test-data/1.txt
</code></pre></div>  </div>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-2" class="box-title"><button type="button" aria-controls="solution-2-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">data = {
    "history_id": new_hist["id"],
    "tool_id": "upload1"
}
with open("1.txt", "rb") as f:
    files = {"files_0|file_data": f}
    r = requests.post(base_url + "/tools", data=data, files=files, headers={"x-api-key": api_key})
ret = r.json()
pprint(ret)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-find-the-dataset-in-your-history" class="box-title" aria-label="question box: Find the dataset in your history"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Find the dataset in your history</div>
<p>Find the new uploaded dataset, either from the dict returned by the POST request above or from the history contents.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-3" class="box-title"><button type="button" aria-controls="solution-3-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">hda = ret['outputs'][0]
pprint(hda)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-import-a-workflow" class="box-title" aria-label="question box: Import a workflow"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Import a workflow</div>
<p><strong>Import a workflow</strong> from the local file <code style="color: inherit">convert_to_tab.ga</code> by making a <code style="color: inherit">POST</code> request to <code style="color: inherit">/api/workflows</code>. The only needed data is <code style="color: inherit">workflow</code>, which must be a deserialized JSON representation of the workflow.</p>
<p>You can obtain the <code style="color: inherit">convert_to_tab.ga</code> file from the following URL, you’ll need to download it first.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">https://raw.githubusercontent.com/nsoranzo/bioblend-tutorial/main/test-data/convert_to_tab.ga
</code></pre></div>  </div>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-4" class="box-title"><button type="button" aria-controls="solution-4-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">with open("convert_to_tab.ga", "r") as f:
    workflow_json = json.load(f)
data = {'workflow': workflow_json}
r = requests.post(base_url + "/workflows", data=json.dumps(data), headers=headers)
wf = r.json()
pprint(wf)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-view-the-workflow-details" class="box-title" aria-label="question box: View the workflow details"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: View the workflow details</div>
<p>View the details of the imported workflow by making a GET request to <code style="color: inherit">/api/workflows</code>.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-5" class="box-title"><button type="button" aria-controls="solution-5-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">r = requests.get(base_url + "/workflows/" + wf["id"], headers=headers)
wf = r.json()
pprint(wf)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-invoke-the-workflow" class="box-title" aria-label="question box: Invoke the workflow"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Invoke the workflow</div>
<p><strong>Run</strong> the imported workflow on the uploaded dataset <strong>inside the same history</strong> by making a <code style="color: inherit">POST</code> request to <code style="color: inherit">/api/workflows/WORKFLOW_ID/invocations</code>. The only needed data are <code style="color: inherit">history</code> and <code style="color: inherit">inputs</code>.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-6" class="box-title"><button type="button" aria-controls="solution-6-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">inputs = {0: {'id': hda['id'], 'src': 'hda'}}
data = {
    'history': 'hist_id=' + new_hist['id'],
    'inputs': inputs}
r = requests.post(base_url + "/workflows/" + wf["id"] + "/invocations", data=json.dumps(data), headers=headers)
pprint(r.json())
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-view-the-results" class="box-title" aria-label="question box: View the results"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: View the results</div>
<p>View the results on the Galaxy server with your web browser. Were you successful? Did it run?</p>
</blockquote>
<h1 id="interacting-with-histories-in-bioblend">Interacting with histories in BioBlend</h1>
<p><strong>You need to insert the API key for your Galaxy server in the cell below</strong>:</p>
<ol>
<li>Open the Galaxy server in another browser tab</li>
<li>Click on “User” on the top menu, then “Preferences”</li>
<li>Click on “Manage API key”</li>
<li>Generate an API key if needed, then copy the alphanumeric string and paste it as the value of the <code style="color: inherit">api_key</code> variable below.</li>
</ol>
<p>The user interacts with a Galaxy server through a <code style="color: inherit">GalaxyInstance</code> object:</p>


In [ ]:
from pprint import pprint

import bioblend.galaxy

server = 'https://usegalaxy.eu/'
api_key = ''
gi = bioblend.galaxy.GalaxyInstance(url=server, key=api_key)

<p>The <code style="color: inherit">GalaxyInstance</code> object gives you access to the various controllers, i.e. the resources you are dealing with, like <code style="color: inherit">histories</code>, <code style="color: inherit">tools</code> and <code style="color: inherit">workflows</code>.
Therefore, method calls will have the format <code style="color: inherit">gi.controller.method()</code>. For example, the call to retrieve all histories owned by the current user is:</p>


In [ ]:
pprint(gi.histories.get_histories())

<p>As you can see, methods in BioBlend do not return JSON strings, but <strong>deserialize</strong> them into Python data structures. In particular, <code style="color: inherit">get_</code> methods return a list of dictionaries.</p>
<p>Each dictionary gives basic info about a resource, e.g. for a history you have:</p>
<ul>
<li><code style="color: inherit">id</code>: the unique <strong>identifier</strong> of the history, needed for all specific requests about this resource</li>
<li><code style="color: inherit">name</code>: the name of this history as given by the user</li>
<li><code style="color: inherit">deleted</code>: whether the history has been deleted.</li>
</ul>
<p><strong>New resources</strong> are created with <code style="color: inherit">create_</code> methods, e.g. the call to create a new history is:</p>


In [ ]:
new_hist = gi.histories.create_history(name='BioBlend test')
pprint(new_hist)

<p>As you can see, to make POST requests in BioBlend it is <strong>not necessary to serialize data</strong>, you just pass them explicitly as parameters. The return value is a dictionary with detailed info about the created resource.</p>
<p><code style="color: inherit">get_</code> methods usually have <strong>filtering</strong> capabilities, e.g. it is possible to filter histories <strong>by name</strong>:</p>


In [ ]:
pprint(gi.histories.get_histories(name='BioBlend test'))

<p>To <strong>upload</strong> the local file <code style="color: inherit">1.txt</code> to the new history, you can run the special upload tool by calling the <code style="color: inherit">upload_file</code> method of the <code style="color: inherit">tools</code> controller.</p>
<p>You can obtain the <code style="color: inherit">1.txt</code> file from the following URL, you’ll need to download it first.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">https://raw.githubusercontent.com/nsoranzo/bioblend-tutorial/main/test-data/1.txt
</code></pre></div></div>


In [ ]:
hist_id = new_hist["id"]
pprint(gi.tools.upload_file("1.txt", hist_id))

<p>If you are interested in more <strong>details</strong> about a given resource for which you know the id, you can use the corresponding <code style="color: inherit">show_</code> method. For example, to the get more info for the history we have just populated:</p>


In [ ]:
pprint(gi.histories.show_history(history_id=hist_id))

<p>As you can see, there are much more entries in the returned dictionary, e.g.:</p>
<ul>
<li><code style="color: inherit">create_time</code></li>
<li><code style="color: inherit">size</code>: total disk space used by the history</li>
<li><code style="color: inherit">state_ids</code>: ids of history datasets for each possible state.</li>
</ul>
<p>To get the list of <strong>datasets contained</strong> in a history, simply add <code style="color: inherit">contents=True</code> to the previous call.</p>


In [ ]:
hdas = gi.histories.show_history(history_id=hist_id, contents=True)
pprint(hdas)

<p>The dictionaries returned when showing the history content give basic info about each dataset, e.g.: <code style="color: inherit">id</code>, <code style="color: inherit">name</code>, <code style="color: inherit">deleted</code>, <code style="color: inherit">state</code>, <code style="color: inherit">url</code>…</p>
<p>To get the details about a specific dataset, you can use the <code style="color: inherit">datasets</code> controller:</p>


In [ ]:
hda0_id = hdas[0]['id']
print(hda0_id)
pprint(gi.datasets.show_dataset(hda0_id))

<p>Some of the interesting additional dictionary entries are:</p>
<ul>
<li><code style="color: inherit">create_time</code></li>
<li><code style="color: inherit">creating job</code>: id of the job which created this dataset</li>
<li><code style="color: inherit">download_url</code>: URL to download the dataset</li>
<li><code style="color: inherit">file_ext</code>: the Galaxy data type of this dataset</li>
<li><code style="color: inherit">file_size</code></li>
<li><code style="color: inherit">genome_build</code>: the genome build (dbkey) associated to this dataset.</li>
</ul>
<p>To <strong>update</strong> a resource, use the <code style="color: inherit">update_</code> method, e.g. to change the name of the new history:</p>


In [ ]:
pprint(gi.histories.update_history(new_hist['id'], name='Updated history'))

<p>The return value of <code style="color: inherit">update_</code> methods is usually a dictionary with detailed info about the updated resource.</p>
<p>Finally to <strong>delete</strong> a resource, use the <code style="color: inherit">delete_</code> method, e.g.:</p>


In [ ]:
pprint(gi.histories.delete_history(new_hist['id']))

<h2 id="exercise-bioblend">Exercise: BioBlend</h2>
<p><strong>Goal</strong>: Upload a file to a new history, import a workflow and run it on the uploaded dataset.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-initialise-1" class="box-title" aria-label="question box: Initialise"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Initialise</div>
<p>Create a <code style="color: inherit">GalaxyInstance</code> object.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-7" class="box-title"><button type="button" aria-controls="solution-7-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">from pprint import pprint

import bioblend.galaxy

server = 'https://usegalaxy.eu/'
api_key = ''
gi = bioblend.galaxy.GalaxyInstance(url=server, key=api_key)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-new-history-1" class="box-title" aria-label="question box: New History"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: New History</div>
<p>Create a new Galaxy history.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-8" class="box-title"><button type="button" aria-controls="solution-8-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">new_hist = gi.histories.create_history(name='New history')
pprint(new_hist)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-upload-a-dataset-1" class="box-title" aria-label="question box: Upload a dataset"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Upload a dataset</div>
<p><strong>Upload</strong> the local file <code style="color: inherit">1.txt</code> to the new history using <code style="color: inherit">tools.upload_file()</code> .</p>
<p>You can obtain the <code style="color: inherit">1.txt</code> file from the following URL, you’ll need to download it first.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">https://raw.githubusercontent.com/nsoranzo/bioblend-tutorial/main/test-data/1.txt
</code></pre></div>  </div>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-9" class="box-title"><button type="button" aria-controls="solution-9-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">ret = gi.tools.upload_file("1.txt", new_hist["id"])
pprint(ret)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-find-the-dataset-in-your-history-1" class="box-title" aria-label="question box: Find the dataset in your history"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Find the dataset in your history</div>
<p>Find the new uploaded dataset, either from the dict returned by <code style="color: inherit">tools.upload_file()</code> or from the history contents.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-10" class="box-title"><button type="button" aria-controls="solution-10-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">hda = ret['outputs'][0]
pprint(hda)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-import-a-workflow-1" class="box-title" aria-label="question box: Import a workflow"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Import a workflow</div>
<p><strong>Import a workflow</strong> from the local file <code style="color: inherit">convert_to_tab.ga</code> using <code style="color: inherit">workflows.import_workflow_from_local_path()</code> .</p>
<p>You can obtain the <code style="color: inherit">convert_to_tab.ga</code> file from the following URL, you’ll need to download it first.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">https://raw.githubusercontent.com/nsoranzo/bioblend-tutorial/main/test-data/convert_to_tab.ga
</code></pre></div>  </div>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-11" class="box-title"><button type="button" aria-controls="solution-11-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">wf = gi.workflows.import_workflow_from_local_path("convert_to_tab.ga")
pprint(wf)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-view-the-workflow-details-1" class="box-title" aria-label="question box: View the workflow details"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: View the workflow details</div>
<p>View the details of the imported workflow using <code style="color: inherit">workflows.show_workflow()</code></p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-12" class="box-title"><button type="button" aria-controls="solution-12-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">wf = gi.workflows.show_workflow(wf['id'])
pprint(wf)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-invoke-the-workflow-1" class="box-title" aria-label="question box: Invoke the workflow"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Invoke the workflow</div>
<p><strong>Run</strong> the imported workflow on the uploaded dataset <strong>inside the same history</strong> using <code style="color: inherit">workflows.invoke_workflow()</code> .</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-13" class="box-title"><button type="button" aria-controls="solution-13-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">inputs = {0: {'id': hda['id'], 'src': 'hda'}}
ret = gi.workflows.invoke_workflow(wf['id'], inputs=inputs, history_id=new_hist['id'])
pprint(ret)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-view-the-results-1" class="box-title" aria-label="question box: View the results"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: View the results</div>
<p>View the results on the Galaxy server with your web browser. Were you successful? Did it run?</p>
</blockquote>
<h1 id="interacting-with-histories-in-bioblendobjects">Interacting with histories in BioBlend.objects</h1>
<p><strong>You need to insert the API key for your Galaxy server in the cell below</strong>:</p>
<ol>
<li>Open the Galaxy server in another browser tab</li>
<li>Click on “User” on the top menu, then “Preferences”</li>
<li>Click on “Manage API key”</li>
<li>Generate an API key if needed, then copy the alphanumeric string and paste it as the value of the <code style="color: inherit">api_key</code> variable below.</li>
</ol>
<p>The user interacts with a Galaxy server through a <code style="color: inherit">GalaxyInstance</code> object:</p>


In [ ]:
from pprint import pprint

import bioblend.galaxy.objects

server = 'https://usegalaxy.eu/'
api_key = ''
gi = bioblend.galaxy.objects.GalaxyInstance(url=server, api_key=api_key)

<p>All <code style="color: inherit">GalaxyInstance</code> method calls have the <code style="color: inherit">client.method()</code> format, where <code style="color: inherit">client</code> is the name of the resources you dealing with. There are 2 methods to get the list of resources:</p>
<ul>
<li><code style="color: inherit">get_previews()</code>: lightweight (one GET request), retrieves basic resources’ info, returns a list of <strong>preview</strong> objects</li>
<li><code style="color: inherit">list()</code>: one GET request for each resource, retrieves full resources’ info, returns a list of <strong>full</strong> objects.</li>
</ul>
<p>For example, the call to retrieve previews of all histories owned by the current user is:</p>


In [ ]:
pprint(gi.histories.get_previews())

<p><strong>New resources</strong> are created with <code style="color: inherit">create()</code> methods, e.g. to create a new history:</p>


In [ ]:
new_hist = gi.histories.create(name='BioBlend test')
new_hist

<p>As you can see, the <code style="color: inherit">create()</code> methods in BioBlend.objects returns an object, not a dictionary.</p>
<p>Both <code style="color: inherit">get_previews()</code> and <code style="color: inherit">list()</code> methods usually have <strong>filtering</strong> capabilities, e.g. it is possible to filter histories <strong>by name</strong>:</p>


In [ ]:
pprint(gi.histories.list(name='BioBlend test'))

<p>To <strong>upload</strong> the local file <code style="color: inherit">1.txt</code> to the new history, you can run the special upload tool by calling the <code style="color: inherit">upload_file</code> method of the <code style="color: inherit">History</code> object.</p>
<p>You can obtain the <code style="color: inherit">1.txt</code> file from the following URL, you’ll need to download it first.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">https://raw.githubusercontent.com/nsoranzo/bioblend-tutorial/main/test-data/1.txt
</code></pre></div></div>


In [ ]:
hda = new_hist.upload_file("1.txt")
hda

<p>Please note that with BioBlend.objects there is no need to find the upload dataset, since <code style="color: inherit">upload_file()</code> already returns a <code style="color: inherit">HistoryDatasetAssociation</code> object.</p>
<p>Both <code style="color: inherit">HistoryPreview</code> and <code style="color: inherit">History</code> objects have many of their properties available as <strong>attributes</strong>, e.g. the id.</p>
<p>If you need to specify the unique <strong>id</strong> of the resource to retrieve, you can use the <code style="color: inherit">get()</code> method, e.g. to get back the history we created before:</p>


In [ ]:
gi.histories.get(new_hist.id)

<p>To get the list of <strong>datasets contained</strong> in a history, simply look at the <code style="color: inherit">content_infos</code> attribute of the <code style="color: inherit">History</code> object.</p>


In [ ]:
pprint(new_hist.content_infos)

<p>To get the details about one dataset, you can use the <code style="color: inherit">get_dataset()</code> method of the <code style="color: inherit">History</code> object:</p>


In [ ]:
new_hist.get_dataset(hda.id)

<p>You can also filter history datasets by name using the <code style="color: inherit">get_datasets()</code> method of <code style="color: inherit">History</code> objects.</p>
<p>To <strong>update</strong> a resource, use the <code style="color: inherit">update()</code> method of its object, e.g. to change the history name:</p>


In [ ]:
new_hist.update(name='Updated history')

<p>The return value of <code style="color: inherit">update()</code> methods is the updated object.</p>
<p>Finally to <strong>delete</strong> a resource, you can use the <code style="color: inherit">delete()</code> method of the object, e.g.:</p>


In [ ]:
new_hist.delete()

<h2 id="exercise-bioblendobjects">Exercise: BioBlend.objects</h2>
<p><strong>Goal</strong>: Upload a file to a new history, import a workflow and run it on the uploaded dataset.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-initialise-2" class="box-title" aria-label="question box: Initialise"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Initialise</div>
<p>Create a <code style="color: inherit">GalaxyInstance</code> object.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-14" class="box-title"><button type="button" aria-controls="solution-14-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">from pprint import pprint

import bioblend.galaxy

server = 'https://usegalaxy.eu/'
api_key = ''
gi = bioblend.galaxy.objects.GalaxyInstance(url=server, api_key=api_key)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-new-history-2" class="box-title" aria-label="question box: New History"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: New History</div>
<p>Create a new Galaxy history.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-15" class="box-title"><button type="button" aria-controls="solution-15-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">new_hist = gi.histories.create(name='New history')
new_hist
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-upload-a-dataset-2" class="box-title" aria-label="question box: Upload a dataset"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Upload a dataset</div>
<p><strong>Upload</strong> the local file <code style="color: inherit">1.txt</code> to the new history using the <code style="color: inherit">upload_file()</code> method of <code style="color: inherit">History</code> objects.</p>
<p>You can obtain the <code style="color: inherit">1.txt</code> file from the following URL, you’ll need to download it first.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">https://raw.githubusercontent.com/nsoranzo/bioblend-tutorial/main/test-data/1.txt
</code></pre></div>  </div>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-16" class="box-title"><button type="button" aria-controls="solution-16-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">hda = new_hist.upload_file("1.txt")
hda
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-import-a-workflow-2" class="box-title" aria-label="question box: Import a workflow"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Import a workflow</div>
<p><strong>Import a workflow</strong> from the local file <code style="color: inherit">convert_to_tab.ga</code> using <code style="color: inherit">workflows.import_new()</code></p>
<p>You can obtain the <code style="color: inherit">convert_to_tab.ga</code> file from the following URL, you’ll need to download it first.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">https://raw.githubusercontent.com/nsoranzo/bioblend-tutorial/main/test-data/convert_to_tab.ga
</code></pre></div>  </div>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-17" class="box-title"><button type="button" aria-controls="solution-17-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">with open("convert_to_tab.ga", "r") as f:
    wf_string = f.read()
wf = gi.workflows.import_new(wf_string)
wf
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-view-the-workflow-inputs" class="box-title" aria-label="question box: View the workflow inputs"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: View the workflow inputs</div>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-18" class="box-title"><button type="button" aria-controls="solution-18-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">wf.inputs
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-invoke-the-workflow-2" class="box-title" aria-label="question box: Invoke the workflow"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: Invoke the workflow</div>
<p><strong>Run</strong> the imported workflow on the uploaded dataset <strong>inside the same history</strong> using the <code style="color: inherit">invoke()</code> method of <code style="color: inherit">Workflow</code> objects.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div id="solution-19" class="box-title"><button type="button" aria-controls="solution-19-contents" aria-expanded="true" aria-label="Toggle solution box: "><i class="far fa-eye" aria-hidden="true"></i><span class="visually-hidden"></span> Solution<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">inputs = {'0': hda}
wf.invoke(inputs=inputs, history=new_hist)
</code></pre></div>    </div>
</details>
</blockquote>


In [ ]:
# Try it out here!


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div id="question-view-the-results-2" class="box-title" aria-label="question box: View the results"><i class="far fa-question-circle" aria-hidden="true"></i><span class="visually-hidden"></span> Question: View the results</div>
<p>View the results on the Galaxy server with your web browser. Were you successful? Did it run?</p>
</blockquote>
<h1 id="optional-extra-exercises">Optional Extra Exercises</h1>
<p>If you have completed the exercise, you can try to perform these extra tasks with the help of the online documentation:</p>
<ol>
<li>Download the workflow result to your computer</li>
<li>Publish your history</li>
</ol>


# Key Points

- The API allows you to use Galaxy's capabilities programmatically.
- BioBlend makes using the Galaxy API from Python easier.
- BioBlend objects is an object-oriented interface for interacting with Galaxy.

# Congratulations on successfully completing this tutorial!

Please [fill out the feedback on the GTN website](https://training.galaxyproject.org/training-material/topics/dev/tutorials/bioblend-api/tutorial.html#feedback) and check there for further resources!
